In [130]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [163]:
train = pd.read_csv('train.csv')
testA_file = pd.read_csv('testA.csv')

In [162]:
for c,dtype in zip(train.columns,train.dtypes):
    if dtype != 'float64' and dtype !='int64' and c !='ID':
        print (c,dtype)
        train[c] = LabelEncoder().fit_transform(train[c])
        testA_file[c] = LabelEncoder().fit_transform(testA_file[c])

In [133]:
missing_df = train.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
#missing_df = missing_df.ix[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')


In [134]:
drop_list = missing_df[missing_df['missing_count']==500].column_name.tolist()
drop_list.append('ID')

In [135]:
train = train.drop(drop_list,axis=1)
testA = testA_file.drop(drop_list,axis=1)

In [137]:
for c in train.columns:
    train[c] = train[c].fillna(train[c].median())
for c in testA.columns:
    testA[c] = testA[c].fillna(testA[c].median())

In [150]:
x_train = train.drop(['Y'],axis=1)
x_test = testA.as_matrix()
y_train = train['Y'].values

In [151]:
x_train = x_train.values.astype(np.float32,copy=False)
testA = testA.values.astype(np.float32,copy=False)

In [153]:
testA

array([[  0.00000000e+00,   1.00849998e+02,   4.23000008e-01, ...,
          7.39999989e-04,   7.39999989e-04,   2.39999989e+12],
       [  0.00000000e+00,   1.00800003e+02,   3.21999997e-01, ...,
          6.60000020e-04,   6.60000020e-04,   2.39999989e+12],
       [  5.00000000e+00,   1.00750000e+02,   5.26000023e-01, ...,
          7.69999984e-04,   7.69999984e-04,   2.39999989e+12],
       ..., 
       [  4.00000000e+00,   1.02300003e+02,   5.04999995e-01, ...,
          6.79999997e-04,   6.79999997e-04,   2.39999989e+12],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          7.50000007e-04,   7.50000007e-04,   2.39999989e+12],
       [  2.00000000e+00,   9.98600006e+01,   5.61999977e-01, ...,
          6.20000006e-04,   6.20000006e-04,   2.39999989e+12]], dtype=float32)

In [121]:
xgb_params = {}
xgb_params['eta'] = 0.037
xgb_params['max_depth'] = 5
xgb_params['subsample'] = 0.80
xgb_params['objective'] = 'reg:linear'
xgb_params['eval_metric'] = 'mae'
xgb_params['lambda'] = 0.8
xgb_params['alpha'] = 0.4
xgb_params['base_score'] = np.mean(y_train)
xgb_params['silent'] = 1

In [154]:
dtrain = xgb.DMatrix(x_train,y_train)
dtest = dtest = xgb.DMatrix(x_test)

In [155]:
model = xgb.train(xgb_params,dtrain,num_boost_round=250)

In [156]:
xgb_pred = model.predict(dtest)

In [165]:
submission = pd.DataFrame({'ID':testA_file['ID'],'predict':xgb_pred})

In [167]:
submission.to_csv('submission.csv',index=False,header=False)

In [166]:
submission

,ID,predict
0,ID716,2.629385
1,ID717,2.733318
2,ID719,2.913573
3,ID720,3.004717
4,ID721,2.844112
5,ID722,2.785713
6,ID723,2.862635
7,ID724,2.776035
8,ID725,2.800653
9,ID727,2.793876
